In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import numpy as np
import time
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")


No GPU available. Training will run on CPU.


In [2]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=360):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [3]:
def freeze_bert_layers(model, num_layers_to_freeze):
    """
    Freezes the specified number of layers from the bottom of the BERT model.
    Args:
        model: The BERT model
        num_layers_to_freeze: Number of layers to freeze (counting from bottom)
    """
    # Freeze embeddings
    for param in model.bert.embeddings.parameters():
        param.requires_grad = False

    # Freeze the specified number of encoder layers
    for layer in model.bert.encoder.layer[:num_layers_to_freeze]:
        for param in layer.parameters():
            param.requires_grad = False

    # Print trainable parameters info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    print(f"Percentage of trainable parameters: {100 * trainable_params / total_params:.2f}%")



In [4]:
def train_model(model, train_loader, val_loader, device, num_epochs=3):
    # Initialize optimizer only with parameters that require gradients
    optimizer = AdamW(
        [p for p in model.parameters() if p.requires_grad],
        lr=2e-5
    )

    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        total_train_loss = 0
        right_predictions = 0
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            model.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            # Calculate accuracy
            preds = torch.argmax(outputs.logits, dim=1)
            right_predictions += torch.sum(preds == labels).item()

            loss = outputs.loss
            total_train_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        # Validation
        model.eval()
        total_val_loss = 0
        predictions = []
        true_labels = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=labels
                )

                loss = outputs.loss
                total_val_loss += loss.item()

                preds = torch.argmax(outputs.logits, dim=1)
                predictions.extend(preds.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = total_val_loss / len(val_loader)
        accuracy = np.mean(np.array(predictions) == np.array(true_labels))
        end_time = time.time()

        print(f'Epoch {epoch + 1}:')
        print(f'Average training loss: {avg_train_loss:.4f}')
        print(f'Average validation loss: {avg_val_loss:.4f}')
        print(f'Right predictions: {right_predictions} out of {len(train_loader) * 32}')
        print(f'Validation Accuracy: {accuracy:.4f}')
        print(f'Time taken for epoch: {end_time - start_time:.2f} seconds')
        print('-' * 60)


In [6]:
!pip install onnx
!pip install tempfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 36.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement tempfile (from versions: none)
ERROR: No matching distribution found for tempfile


In [7]:
import tempfile, onnx


def convert_pytorch_to_onnx_with_tokenizer(model, tokenizer, max_length=128, onnx_file_path=None):
    """
    Converts a PyTorch model to ONNX format, using tokenizer output as input.

    Args:
    model (torch.nn.Module): The PyTorch model to be converted.
    tokenizer: The tokenizer used to preprocess the input.
    onnx_file_path (str): The file path where the ONNX model will be saved.
    max_length (int): Maximum sequence length for the tokenizer.

    Returns:
    None
    """
    model.eval()

    # Prepare dummy input using the tokenizer
    dummy_input = "This is a sample input text for ONNX conversion."
    inputs = tokenizer(
        dummy_input,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

    # # Get the input names
    input_names = list(inputs.keys())
    input_names = ["input_ids", "attention_mask"]
    print(f"Input names: {input_names}")

    # # Create dummy inputs for ONNX export
    # dummy_inputs = tuple(encoded_input[name] for name in input_names)
    if onnx_file_path is None:
        onnx_file_path = tempfile.mktemp(suffix=".onnx")
    dynamic_axes = {name: {0: "batch_size"} for name in input_names}
    dynamic_axes.update({f"logits": {0: "batch_size"}})
    print(f"dynamic_axes: {dynamic_axes}")
    # Export the model
    torch.onnx.export(
        model,  # model being run
        tuple(inputs[k] for k in input_names),  # model inputs
        onnx_file_path,  # where to save the model
        export_params=True,  # store the trained parameter weights inside the model file
        opset_version=20,  # the ONNX version to export the model to
        do_constant_folding=True,  # whether to execute constant folding for optimization
        input_names=input_names,  # the model's input names
        output_names=["logits"],  # the model's output names
        dynamic_axes=dynamic_axes,
    )  # variable length axes

    print(f"Model exported to {onnx_file_path}")

    # Verify the exported model
    onnx_model = onnx.load(onnx_file_path)
    onnx.checker.check_model(onnx_model)
    print("ONNX model is valid.")
    return onnx_file_path, input_names




In [8]:
from transformers import AutoModel
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=3
# )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [1]:
from transformers import BertConfig

config = BertConfig.from_pretrained(
        model_name,
        num_labels=3,
        hidden_act="gelu_fast"  # or "erf" depending on what you want
    )

model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        config=config,

    )

NameError: name 'BertConfig' is not defined

In [10]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00


In [11]:
import os
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Assume CustomDataset and train_model are defined elsewhere in your code
# CustomDataset should handle tokenization

def main(model, tokenizer):
    dataset = load_dataset("takala/financial_phrasebank", 'sentences_allagree')

    train_texts = dataset['train']['sentence']
    train_labels = dataset['train']['label']

    train_texts, val_texts, train_labels, val_labels = train_test_split(
        train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
    )

    # Create datasets
    train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
    val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=32, num_workers=2)

    # Set device and move data to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Train the model
    train_model(model, train_loader, val_loader, device, num_epochs=3)

    # Make model tensors contiguous and move to CPU before saving
    model = model.cpu()

    # Save the fine-tuned model as an ONNX file
    onnx_file_path, input_names = convert_pytorch_to_onnx_with_tokenizer(
        model, tokenizer, max_length=128, onnx_file_path="./my_new_bert_model.onnx"
    )
    print(f"ONNX file path: {onnx_file_path}")
    print(f"Input names: {input_names}")

    # Test the model on a few examples
    model.eval()
    test_texts = [
        "This movie was absolutely fantastic! I loved every minute of it.",
        "Terrible waste of time. The plot made no sense and the acting was awful."
    ]

    with torch.no_grad():
        inputs = tokenizer(
            test_texts,
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)

        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=1)

        for text, pred in zip(test_texts, predictions):
            sentiment = "Positive" if pred[1] > pred[0] else "Negative"
            confidence = max(pred).item()  # Get confidence for the highest score
            print(f"\nText: {text}")
            print(f"Sentiment: {sentiment} (confidence: {confidence:.2%})")

# Call the main function
# Assuming model and tokenizer are already defined
main(model, tokenizer)


README.md:   0%|          | 0.00/8.88k [00:00<?, ?B/s]

financial_phrasebank.py:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

The repository for takala/financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/takala/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1:
Average training loss: 0.9973
Average validation loss: 0.9352
Right predictions: 1082 out of 1824
Validation Accuracy: 0.6137
Time taken for epoch: 88.50 seconds
------------------------------------------------------------
Epoch 2:
Average training loss: 0.9227
Average validation loss: 0.8935
Right predictions: 1114 out of 1824
Validation Accuracy: 0.6137
Time taken for epoch: 84.76 seconds
------------------------------------------------------------
Epoch 3:
Average training loss: 0.8991
Average validation loss: 0.8813
Right predictions: 1113 out of 1824
Validation Accuracy: 0.6137
Time taken for epoch: 89.29 seconds
------------------------------------------------------------
Input names: ['input_ids', 'attention_mask']
dynamic_axes: {'input_ids': {0: 'batch_size'}, 'attention_mask': {0: 'batch_size'}, 'logits': {0: 'batch_size'}}
Model exported to ./my_new_bert_model.onnx
ONNX model is valid.
ONNX file path: ./my_new_bert_model.onnx
Input names: ['input_ids', 'attention_mas